# Problem set 6: Walras-equilibrium models

**Table of contents**<a id='toc0_'></a>    
- 1. [Simple supply and demand](#toc1_)    
- 2. [Exchange economy with quasi-linear preferences](#toc2_)    
  - 2.1. [Improvement set](#toc2_1_)    
  - 2.2. [Equilibrium](#toc2_2_)    
  - 2.3. [Dictators](#toc2_3_)    
- 3. [Production Economy with CO2 tax](#toc3_)    
  - 3.1. [Labor supply](#toc3_1_)    
  - 3.2. [Grid search](#toc3_2_)    
  - 3.3. [Equilibrium prices](#toc3_3_)    
  - 3.4. [Optimal CO2 tax](#toc3_4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'axes.grid':True,'grid.color':'black','grid.alpha':'0.25','grid.linestyle':'--'})
plt.rcParams.update({'font.size': 14})
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
from scipy import optimize

## 1. <a id='toc1_'></a>[Simple supply and demand](#toc0_)

In a simple market model, the **demand function** gives the quantity of a good that consumers are willing to buy
at each price, while the **supply function** gives the quantity that producers are willing to sell.

An **equilibrium** occurs when the quantity demanded equals the quantity supplied:

$$
D(p^*) = S(p^*)
$$

or equivalently when **excess demand** $ Z(p) = D(p) - S(p) $ equals zero.

The equilibrium price $ p^* $ is found by solving:

$$
Z(p) = 0
$$

Once $ p^* $ is found, the corresponding equilibrium quantity is $ q^* = D(p^*) = S(p^*) $.

Here are the demand, supply and excess demand functions:

In [3]:
def demand(p, a=10, b=1):
    """Demand: D(p) = a - b*p"""
    return a - b * p

def supply(p, c=2, d=1):
    """Supply: S(p) = c + d*p"""
    return c + d * p

def excess_demand(p,do_print=False):
    """Excess demand function"""

    Z = demand(p) - supply(p)
    if do_print: print(f'p = {p:12.8f}, {Z = :12.8f}')

    return Z

**Question a:** Find the equilibrium price with a root-finder.

In [4]:
obj = lambda p: excess_demand(p,do_print=True)
# res = ADD CODE HERE#

# p_star = res.root
# q_star = demand(p_star)

# print(f'Equilibrium price: p* = {p_star:.2f}')
# print(f'Equilibrium quantity: q* = {q_star:.2f}')

**Question b:** Find the equilibrium price applying the following auction-like algorithm (also called tâtonnement).

1. Guess on $p = 5.0$ and set $i=0$
1. Calculate $Z(p)$.
1. Stop if $Z(p) < \text{tol.}$.
1. Else set $p = p + \nu Z(p)$ and go to step 2 if $i+1 < \text{maxiter}$.


In [5]:
nu = 0.1
p = 5.0
tol = 1e-8
maxiter = 100
for i in range(maxiter):
    Z = excess_demand(p)
    # ADD CODE HERE
    print(f'Iteration {i+1:3d}: p = {p:12.8f}, Z = {Z:12.8f}')

if abs(Z) < tol:
    print(f'Tâtonnement converged in {i+1} iterations to price p = {p:.8f}')
else:
    print(f'Tâtonnement did not converge in {maxiter} iterations.')

Iteration   1: p =   5.00000000, Z =  -2.00000000
Iteration   2: p =   5.00000000, Z =  -2.00000000
Iteration   3: p =   5.00000000, Z =  -2.00000000
Iteration   4: p =   5.00000000, Z =  -2.00000000
Iteration   5: p =   5.00000000, Z =  -2.00000000
Iteration   6: p =   5.00000000, Z =  -2.00000000
Iteration   7: p =   5.00000000, Z =  -2.00000000
Iteration   8: p =   5.00000000, Z =  -2.00000000
Iteration   9: p =   5.00000000, Z =  -2.00000000
Iteration  10: p =   5.00000000, Z =  -2.00000000
Iteration  11: p =   5.00000000, Z =  -2.00000000
Iteration  12: p =   5.00000000, Z =  -2.00000000
Iteration  13: p =   5.00000000, Z =  -2.00000000
Iteration  14: p =   5.00000000, Z =  -2.00000000
Iteration  15: p =   5.00000000, Z =  -2.00000000
Iteration  16: p =   5.00000000, Z =  -2.00000000
Iteration  17: p =   5.00000000, Z =  -2.00000000
Iteration  18: p =   5.00000000, Z =  -2.00000000
Iteration  19: p =   5.00000000, Z =  -2.00000000
Iteration  20: p =   5.00000000, Z =  -2.00000000


## 2. <a id='toc2_'></a>[Exchange economy with quasi-linear preferences](#toc0_)

We consider an **exchange economy** like the one in the lecture, but with **quasi-linear preferencer**.

The general setup is:

- Two consumers, A and B.
- Two goods, $x_{1}$ and $x_{2}$. 

**Endowments:**

* The initial endowments are $\omega_{1}^{A}\geq0$ and $\omega_{2}^{A}\geq0$. 
* The total endowment of each good is always one, such that $\omega_{1}^{B} = 1-\omega_{1}^{A}$ and $\omega_{2}^{B} = 1-\omega_{2}^{A}$.
* We define the vectors $\boldsymbol{\omega}^{A}$ = $(\omega_{1}^{A},\omega_{2}^{A})$, and $\boldsymbol{\omega}^{B}=(\omega_{1}^{B},\omega_{2}^{B})$.

**Prices:** $\boldsymbol{p}=(p_{1},p_{2})$ with numeraire $p_2 = 1$.

**Utility functions**:

$$
\begin{aligned}
u^{A}(x_{1},x_{2})	&= \log(x_{1}) + {\alpha} x_{2},\,\,\,\alpha>0 \\
u^{B}(x_{1},x_{2})	&= \log(x_{1}) + {\beta} x_{2},\,\,\,\beta>0 \\
\end{aligned}
$$

**Demand functions:** We define the vectors $\boldsymbol{x}^{A}$ = $(x_{1}^{A},x_{2}^{A})$, and $\boldsymbol{x}^{B}=(x_{1}^{B},x_{2}^{B})$.

$$
\begin{aligned}
x^{A\star}(\boldsymbol{p},\boldsymbol{\omega^A}) & =
\begin{cases}
\left(\frac{ p_{2}}{\alpha p_{1}},\frac{m-\frac{p_{2}}{\alpha}}{p_{2}}\right)
& \text{if }m^A>\frac{p_{2}}{\alpha} \\
\left(\frac{m}{p_{1}},0\right) 
& \text{if }m^A\leq\frac{p_{2}}{\alpha} 
\end{cases} \\
m^A &= p_1 \omega_1^A + p_2 \omega_2^A 
\end{aligned}
$$

$$
\begin{aligned}
x^{A\star}(\boldsymbol{p},\boldsymbol{\omega^B}) & =
\begin{cases}
\left(\frac{ p_{2}}{\alpha p_{1}},\frac{m-\frac{p_{2}}{\alpha}}{p_{2}}\right)
& \text{if }m^B>\frac{p_{2}}{\alpha} \\
\left(\frac{m}{p_{1}},0\right) 
& \text{if }m^B\leq\frac{p_{2}}{\alpha} 
\end{cases} \\
m^B &= p_1 \omega_1^B + p_2 \omega_2^B
\end{aligned}
$$

**Walras equilibrium**: Market clearing requires demand = endowment (supply),

$$
\begin{aligned}
x_{1}^{A\star}(\boldsymbol{p},\boldsymbol{\omega}^{A})+x_{1}^{B\star}(\boldsymbol{p},\boldsymbol{\omega}^{B}) &= \omega_{1}^{A}+\omega_{1}^{B} \\
x_{2}^{A\star}(\boldsymbol{p},\boldsymbol{\omega}^{A})+x_{2}^{B\star}(\boldsymbol{p},\boldsymbol{\omega}^{B}) &= \omega_{2}^{A}+\omega_{2}^{B}
\end{aligned}
$$ 

The class `ExchangeEconomyModelQuasiLinearClass` in `ExchangeEconomyModelQuasiLinear.py` is the starting point for solving this model:

1. It inherites the basic functationality from `ExchangeEconomyModelClass` in `ExchangeEconomyModel.py`.
2. You will be updating the currently not implemented methods.

In [6]:
from ExchangeEconomyQuasiLinearModel import ExchangeEconomyModelQuasiLinearClass

**Note:** The solution model class is in `A2_ExchangeEconomyQuasiLinearModel.py`.

### 2.1. <a id='toc2_1_'></a>[Improvement set](#toc0_)

Update the `.indifference_curve_A` and `.indifference_curve_B` methods.

Create a figure with:

1. The endowment
1. The improvement set
1. The indifferences curves through the endowment
1. Indifferences curves just below and above the endowment

Remember to use the plotting functionalities in the class.

In [20]:
model = ExchangeEconomyModelQuasiLinearClass()

# par = model.par
# sol = model.sol
# sim = model.sim

# fig,ax_A,ax_B = model.create_edgeworthbox()

# ax_A.scatter(par.w1A,par.w2A,marker='s',color='black',label='endowment',zorder=3)

# model.indifference_curve_A(ax_A,par.w1A,par.w2A,color=colors[0],label='A')
# # ADD MORE CURVES HERE

# model.indifference_curve_B(ax_B,1-par.w1A,1-par.w2A,color=colors[1],label='B')
# # ADD MORE CURVES HERE

# model.plot_improvement_set(ax_A)

# model.add_legend(ax_A,ax_B)

### 2.2. <a id='toc2_2_'></a>[Equilibrium](#toc0_)

**Task:** Update the `.demand_A` and `.demand_B` methods.

**Question A:** Solve for the Walras-equilibrium.

In [8]:
from A2_ExchangeEconomyQuasiLinearModel import ExchangeEconomyQuasiLineaModelClass
model = ExchangeEconomyQuasiLineaModelClass()

In [9]:
# model.solve_walras(p_guess=1.0,print_output=True)

**Question B:** Show how the equilibrium price and allocation varies with $\alpha \in [0.5,3.0]$.

In [10]:
# for alpha in np.linspace(0.5,3.0,20):
#     model_ = ExchangeEconomyQuasiLineaModelClass()
#     model_.par.alpha = alpha
#     # ADD YOUR CODE HERE

**Question C:** Find the equilibrium when $\alpha = 0.1$.

In [11]:
# model_ = ExchangeEconomyQuasiLineaModelClass()
# model_.par.alpha = 0.1

# try:
#     model_.solve_walras(p_guess=1.0,print_output=False)
# except:
#     print(f'No convergence for alpha = {model_.par.alpha:.2f}')

# # ADD SOME CODE HERE

# print('Trying again...')
# model_.solve_walras(p_guess=1.0,print_output=True)

### 2.3. <a id='toc2_3_'></a>[Dictators](#toc0_)

When A is dictator, A maximizes own utility:

$$
\max_{(x_{1}^{A},x_{2}^{A})\in[0,1]\times[0,1]}u^{A}(x_{1}^{A},x_{2}^{A})	
$$

subject to B utility not being worse than with her endowment,

$$
u^{B}(1-x_{1}^{A},1-x_{2}^{A})\geq u^{B}(\omega_{1}^{B},\omega_{2}^{B})
$$

**Question A:** Solve the dictator problem for A and plot the solution together with the endowment, the improvement set, and the equilibrium.

In [12]:
# write your code here

**Question B:** Solve the dictator problem for B and plot the solution the endowment, the improvement set, the equilibrium and the outcome when A is dictator.

In [13]:
# write your code here

## 3. <a id='toc3_'></a>[Production Economy with CO2 tax](#toc0_)

Consider a production economy with **two firms** indexed by $j \in \{1,2\}$.

Each produce its own good, and maximize profits:

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

**Firm 1 is green = no emissions. Firm is black = emissions proportional to output.**

**Optimal firm behavior** is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The **implied profits** are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single **consumer** supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm. She maximizes utility:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax on good 2 and $T$ is a lump-sum transfer.

For a given $\ell$, it can be shown that optimal consumption behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$

**Optimal labor supply** is:

$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$

This implies **optimal consumption** as:

$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$

The **government** chooses $\tau$ and balances its budget so $T=\tau c_2^*$.

We initially set $\tau,T=0$. But the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

where $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Market clearing** requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$

We choose $w=1$ as numeraire.

In [ ]:
from ProductionCO2Model import ProductionCO2ModelClass

**Note:** The solution model class is in `A3_ProductionCO2Model.py`.

### 3.1. <a id='toc3_1_'></a>[Labor supply](#toc0_)

**Task:** Add the numerical optimizer in the `.households` method.

**Question a:** Let $p_1$ vary in `[0.5,1.0,1.5]` and vary $p_2$ in `[0.5,1.0,1.5]`. How does labor supply and consumption change?

In [ ]:
model = ProductionCO2ModelClass()

# model.sol.pi1 = 0.0
# model.sol.pi2 = 0.0
# p2 = 1.0
# for p1 in [0.5,1.0,1.5]:
#     for p2 in [0.5,1.0,1.5]:
#         model.households(p1,p2)
#         print(f'p1 = {p1:0.2f}, p2 = {p2:0.2f}')
#         print(f'households: l = {model.sol.l:0.4f}, c1 = {model.sol.c1:0.4f}, c2 = {model.sol.c2:0.4f}')
#         print()

**Question b:** Let $p_1$ vary in `[0.5,1.0,1.5]` and vary $p_2$ in `[0.5,1.0,1.5]`. How does firm behvior, firm profit and household labor supply and consumption change?

In [16]:
# p2 = 1.0
# for p1 in [0.5,1.0,1.5]:
#     for p2 in [0.5,1.0,1.5]:
#         model.firms(p1,p2)
#         model.households(p1,p2)
#         print(f'p1 = {p1:0.2f}, p2 = {p2:0.2f}')
#         print(f'firms: y1 = {model.sol.y1:0.4f}, y2 = {model.sol.y2:0.4f}, pi1 = {model.sol.pi1:0.4f}, pi2 = {model.sol.pi2:0.4f}')
#         print(f'households: l = {model.sol.l:0.4f}, c1 = {model.sol.c1:0.4f}, c2 = {model.sol.c2:0.4f}')
#         print()

### 3.2. <a id='toc3_2_'></a>[Grid search](#toc0_)

**Task:** Add the required code in the `.market_clearing` method.

**Question:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`, and determine where they are closest to being fulfilled.

In [17]:
# model.solve_grid_search(do_print=True)

### 3.3. <a id='toc3_3_'></a>[Equilibrium prices](#toc0_)

**Task:** Add the required code in the `.solve` method.

**Question:** Find the equilibrium prices $p_1$ and $p_2$.

In [18]:
#check = model.solve(do_print=True)

### 3.4. <a id='toc3_4_'></a>[Optimal CO2 tax](#toc0_)

**Task:** Add the required code in the `.optimal_gov` method.

**Question:** What value of $\tau$ should the government choose to maximize $SWF$?

In [19]:
# model.optimal_gov()